In [25]:
from models import WorkersOrm, ResumesOrm, Workload
from schemas import ResumesDTO, ResumesRelDTO, WorkersDTO, WorkersRelDTO
from database import session_factory, sync_engine

from sqlalchemy import select, func, and_, or_, Integer
from sqlalchemy.orm import selectinload, joinedload

Без relationship

In [31]:
with session_factory() as session:
    query = (
        select(WorkersOrm)
        .limit(2))
    res = session.execute(query)
    result_orm = res.scalars().all()
    print(f"{result_orm}")
    result_dto = [WorkersDTO.model_validate(row, from_attributes=True) for row in result_orm]
    print(f"{result_dto}")

2024-02-21 10:38:35,607 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-21 10:38:35,610 INFO sqlalchemy.engine.Engine SELECT workers.id, workers.username 
FROM workers 
 LIMIT %(param_1)s::INTEGER
2024-02-21 10:38:35,611 INFO sqlalchemy.engine.Engine [cached since 4018s ago] {'param_1': 2}
[<WorkersOrm id=1, username=Jack>, <WorkersOrm id=2, username=Michael>]
[WorkersDTO(username='Jack', id=1), WorkersDTO(username='Michael', id=2)]
2024-02-21 10:38:35,618 INFO sqlalchemy.engine.Engine ROLLBACK


C relationship

In [30]:

with session_factory() as session:
    query = (
        select(WorkersOrm)
        .options(selectinload(WorkersOrm.resumes))
    )
    res = session.execute(query)
    
    result_orm = res.scalars().all()
    result_dto = [WorkersRelDTO.model_validate(row, from_attributes=True) for row in result_orm]
    print(f"{result_dto}")

2024-02-21 10:36:09,431 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-21 10:36:09,434 INFO sqlalchemy.engine.Engine SELECT workers.id, workers.username 
FROM workers
2024-02-21 10:36:09,436 INFO sqlalchemy.engine.Engine [cached since 1803s ago] {}
2024-02-21 10:36:09,443 INFO sqlalchemy.engine.Engine SELECT resumes.worker_id AS resumes_worker_id, resumes.id AS resumes_id, resumes.title AS resumes_title, resumes.compensation AS resumes_compensation, resumes.workload AS resumes_workload, resumes.created_at AS resumes_created_at, resumes.updated_at AS resumes_updated_at 
FROM resumes 
WHERE resumes.worker_id IN (%(primary_keys_1)s::INTEGER, %(primary_keys_2)s::INTEGER, %(primary_keys_3)s::INTEGER, %(primary_keys_4)s::INTEGER, %(primary_keys_5)s::INTEGER)
2024-02-21 10:36:09,444 INFO sqlalchemy.engine.Engine [cached since 3591s ago] {'primary_keys_1': 1, 'primary_keys_2': 2, 'primary_keys_3': 3, 'primary_keys_4': 4, 'primary_keys_5': 5}
[WorkersRelDTO(username='Jack', id=1, resume

JOIN

In [ ]:
with session_factory() as session:
    query = (
        select(
            ResumesOrm.workload,
            func.avg(ResumesOrm.compensation)
            .cast(Integer)
            .label("avg_compensation"),
        )
        .select_from(ResumesOrm)
        .filter(
            and_(
                ResumesOrm.title.contains('Python'),
                ResumesOrm.compensation > 40000,
            )
        )
        .group_by(ResumesOrm.workload)
        .having(func.avg(ResumesOrm.compensation) > 70000)
    )
    print(query.compile(compile_kwargs={"literal_binds": True}))
    res = session.execute(query)
    result = res.all()
    print(f"{result=}")

